In [7]:
pip install gmaps

In [61]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json


# Import API key
from api_keys import g_key

In [9]:
# Load the csv exported in Part I to a DataFrame
vacation_df= pd.read_csv("output_data/cities.csv")
vacation_df.dropna
vacation_df.head(10)

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
5,5,atuona,78,PF,1558378755,76,-9.80,-139.03,81.96,16.49
6,6,flinders,0,AU,1558378755,62,-34.58,150.85,60.01,11.41
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
8,8,tiksi,95,RU,1558378756,88,71.64,128.87,14.10,15.70
9,9,pushkinskiye gory,6,RU,1558378756,70,57.02,28.91,65.04,6.93


In [10]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [12]:
#Use the Lat and Lng as locations and Humidity as the weight.
locations= vacation_df[["Lat","Lng"]].astype(float)
humidity= vacation_df["Humidity"].astype(float)

In [13]:
#Heatmap
fig_1= gmaps.figure()

#Heatmap layer
humidity_layer= gmaps.heatmap_layer(locations,weights= humidity, dissipating=False, max_intensity= 100, point_radius=3)
fig_1.add_layer(humidity_layer)
fig_1

Figure(layout=FigureLayout(height='420px'))

In [14]:
#Narrow down the DataFrame to find your ideal weather condition. 
# Parameters(Ideal temperature is lower than 70 degrees but higher than 60
#Humidity is <60
# Cloudiness less than 20 )

vacation_cities_df= vacation_df.loc[(vacation_df["Max Temp"]>=60)& 
                                    (vacation_df["Max Temp"]<=70)&
                                    (vacation_df["Humidity"]<=60)&
                                    (vacation_df["Cloudiness"]<10)].reset_index(drop=True)               
                                    
vacation_cities_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,30,gobabis,0,NaN,1558378567,24,-22.45,18.97,68.00,2.24
1,166,brainerd,1,US,1558378527,40,46.36,-94.20,63.00,5.82
2,168,straumen,0,NO,1558378786,52,63.87,11.30,68.00,9.17
3,192,port hedland,0,AU,1558378791,51,-20.31,118.58,62.60,6.93
4,214,mount isa,0,AU,1558378794,56,-20.73,139.49,68.00,6.93
5,219,beisfjord,0,NO,1558378796,51,68.38,17.60,60.80,11.41
6,230,moses lake,1,US,1558378804,22,47.13,-119.28,69.80,13.87
7,259,hami,0,CN,1558378834,11,42.84,93.51,69.72,6.20
8,261,fort frances,1,CA,1558378834,38,48.61,-93.40,63.00,9.17
9,262,umtata,0,ZA,1558378835,51,-31.59,28.78,60.80,17.22


Hotel Map
Store into variable named hotel_df.
Add a "Hotel Name" column to the DataFrame.
Set parameters to search for hotels with 5000 meters.
Hit the Google Places API for each city's coordinates.
Store the first Hotel result into the DataFrame.
Plot markers on top of the heatmap.

In [42]:
#base url for nearby search
nearby_url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json?" 

# Create dataframe for hotel
hotel_df= vacation_cities_df.reset_index(drop=True)
hotel_df["Hotel Name"]=""

In [66]:
hotel_df = vacation_cities_df.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Create for loop
for index, row in hotel_df.iterrows():
    
     
    lat = row["Lat"]
    lng = row["Lng"]
    
    
    params["location"] = f"{lat},{lng}"
    
    # Nearby search url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Pull request
    hotel_name = requests.get(base_url, params=params)
    
    # Json hotel_name
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Use try and except method
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing info... continue.")
hotel_df

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -22.5435504,
                    "lng": 19.0741348
                },
                "viewport": {
                    "northeast": {
                        "lat": -22.54220057010728,
                        "lng": 19.07548462989272
                    },
                    "southwest": {
                        "lat": -22.54490022989272,
                        "lng": 19.07278497010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "Sandune Game Lodge",
            "photos": [
                {
                    "height": 22

{
    "html_attributions": [],
    "next_page_token": "Aap_uEB4M4nLx78XVOYQbznC4vsMC7yga1FW9GHZcMBvCM5hmxcMT6iX79PAm3IeQes5MH081JhNrFLeguZdsDdZCWzz0fY3eNnjHE1eJLRduMZuR7e3QFGEyca5JrxmSaglN662eNYWGc6_7vK5f37oSGFXL0Y11YKuZu_syyg2I5WS2eZYr8wOoicSMqwZnC107ovACj6Vcf43uzV8LCW8QVv6MGaVOVm9wKDwEDFHdvWaURBMJDzTZQMZVasRjhAC2h2k9tfNOTxEvVGi2gscIhX3xwCGbssCSOnGqJW55moNTIqpDz4cE2BBQeFmK_JBAkqS-TP65loNk92s2MKzav2B5PpwNZY4yGDApC7A9HDQeK2A6cUenje7Em5vr3sIkuUXiBdS2boHbAzgdwNshtCUaEgC5tEb1nGsMypqOcgIMVPPHaD0Z1l85um8GMnJ",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 46.4944347,
                    "lng": -94.3143575
                },
                "viewport": {
                    "northeast": {
                        "lat": 46.49577464999999,
                        "lng": -94.31229024999998
                    },
                    "southwest": {
                        "lat": 46.4920756

{
    "html_attributions": [],
    "next_page_token": "Aap_uECk17BjM0O1McriKT9A-Dgmccn4OZcsCJ8kbBO55caZEXwCSHyW_sd4n1QwfY4T-VOFLFLlqa0-ltHa5OpUf2B5t0IpyxvnKaUyJqdTj-CHzRTLW-Csu6dd6T_5IJzImcbjGXqGEOuLMqXvyrd0PRiCVMP7hv-z75eFpVVWPhMN_iGVhVhbrN0Jg23Ucs_gq7HGQFUpS3vXDpen9XjZfb0n2ndxLvx5aMUGfnAtfU1gTGUyITsqkDvoYVwFlgNy_hGceTChqU7OyLdsJWi14CS2PFVQzZj_ykb7LWdJM1gZBM7xtyuVK6PTXhoCX1PLTyW31R8WijSt4BxMkYq3gUD5aqOO-ZFNSMKajvWSALpLP28xvJ1Sq2OXn2G9hKjE5yTRuMfM0DkS-z2x4_LIUgEQJWrxWtYOhDACSm-B3B0GT7xI0nCdDQKpoh3v4cAx",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 63.4336324,
                    "lng": 10.4049036
                },
                "viewport": {
                    "northeast": {
                        "lat": 63.43489277989272,
                        "lng": 10.40605482989272
                    },
                    "southwest": {
                        "lat": 63.432193120

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -20.3089027,
                    "lng": 118.6100621
                },
                "viewport": {
                    "northeast": {
                        "lat": -20.30768992010727,
                        "lng": 118.6113451798927
                    },
                    "southwest": {
                        "lat": -20.31038957989271,
                        "lng": 118.6086455201072
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "Hedland Hotel",
            "photos": [
                {
                    "height": 3024,


{
    "html_attributions": [],
    "next_page_token": "Aap_uEAYfLDskTVhTIhcW2yTPs1X7HjMshFLserMATodKM9E2FtQZWMMCoom6Gw6rnaVm15uYHlxvE-QT7Bc5HaH_FAwoJfsG4xqhrPAMOAwHKCzn1IeEahRrHgn6IyCfetbAV5XX_qQkLl8LxvrX_rBNy7YhX3tfs5eq1pVD3_IR9nzHVi3tB-ZaCKUisTEceiDUkNdVzByc3i0ARj_eylqWtiW0GwdTH0-SjkoMRL9IjmZIJ4iG-qaJ0lyfVw672YEduYKY0g8fh6H1CksdWiIByt_lkN0IwrXMQMn7iCulAfrzEObIGPBNIbbYtnvkidG-cXlRmozLNWX793V0VIlhVvMWEaso8sxJi7xYmS23kUOtfy3JHYubguVi2LmyZk5ImGjOpEA45Rgm3W_gb25cXP1039xMBGLAJcP0diwx58JxG0tWCK-RITiAkmPB6OD",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -20.726329,
                    "lng": 139.491381
                },
                "viewport": {
                    "northeast": {
                        "lat": -20.72517322010727,
                        "lng": 139.4928177798927
                    },
                    "southwest": {
                        "lat": -20.7278728

...

{
    "html_attributions": [],
    "next_page_token": "Aap_uEC7d2WDUDZZgLCTn-RYl-0fgsIAMW_TWZry0L3kyoonHVF4bDIGAx_p9e5Rs78m3bp4oHxjimiXVVJJxfyDAKWvfkf757GH3np4QrmvSUQ43qdkVIV-EeSKvSRQZqazm15eZRc496woLJL18U0o4YZ7cKlAQheRIj1xbQPqVfa-u-JM5fkZ5nLqzHXP59iR6CPBXmalJJQw9B4a8sQUAnP5jEZFq6A_BO5bsMBUgPNdzXpPdwy9mFH4eEhs347fAkv8-IKo-1BLZnfD9HDQL8TWhMnEknaSJyAALz6CyFfjoidrdUb1vSlMzqy8oR3psA30kL7rUPQ20zmMqqiw_T16RHPQsZS7Ctlv5yC8pEvooIqzsxcKqHtjmB7TMGVrTATJAl7FI4RtLRKRoQxZt9cxPdRcGB_y9jdSoRk0DogRKFMn3ADMJDy8aZJ__wc3",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 45.6268402,
                    "lng": -1.0511212
                },
                "viewport": {
                    "northeast": {
                        "lat": 45.62815872989272,
                        "lng": -1.049726470107278
                    },
                    "southwest": {
                        "lat": 45.62545907

{
    "html_attributions": [],
    "next_page_token": "Aap_uEBL5DCDl8lhEMTQIs5K2_I4pxoiM9TB5dYBiULq71n33BlwP77b9Or0KMKprA4YhE2OniaceXnhRYR__hz0J96dgVtt-qNJ59O-GNv9gYyGHxBYOjQu7W37UY-RVV2-gpHF6ed_H7Sjnk9nFbC0frov8jt0nb-wJsNYlv46k3OnKfoLZjN_uL4V55_QWhOCHPWlYIfrCa49ceemcqt_4D7fK3pbHNwO6TyS8phxuVppp0QMQyi8mZKR3WX4TSZvzyUAmaJ2ydI-JeGB2uE2UFFmjrpKBb60972wT2QXzzJMcyh7vcejS7YkbZrQMXcNLpHdWRgwKdSbahCbB1p2tS6rMZsolwGchWXUoAPSzaBlV6_aPTJsqXo475C0yOqKzW-khT4YUset_oA62fJi-4EvVLIXk_Q_QIwdvuGbAbKgBDZw2aAWyFvII_36lSTk",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 46.806199,
                    "lng": -100.7826063
                },
                "viewport": {
                    "northeast": {
                        "lat": 46.80761082989272,
                        "lng": -100.7814149701073
                    },
                    "southwest": {
                        "lat": 46.8049111

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 55.4313136,
                    "lng": -116.4719047
                },
                "viewport": {
                    "northeast": {
                        "lat": 55.43302202989272,
                        "lng": -116.4706235201073
                    },
                    "southwest": {
                        "lat": 55.43032237010728,
                        "lng": -116.4733231798927
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "Days Inn by Wyndham High Prairie",
            "opening_hours": {
                "open_now": 

{
    "html_attributions": [],
    "next_page_token": "Aap_uEBPpYgd7mh4oNzCdqq0VjqtnjxcFOrE9tPCAnaUNF2IkPBt381w30VxvBPCW3kCGsA2lEdHxB4p7_w1oVwz8kRAuWXq5BDg0vcT3bx50TX3HOXrjp5YBKD2TEgubYky660VuZBgUBwUQGi_WaWEzxX3jqQAZarjRlO1uGxnm8A2SGM_VO28NKRt38Mi1cfMTBsL3JedK2EU8PSYMZUsGNGXz2CEmV4EYQBupTVfKTbMYd-hdRLclWczBTXdT82W_4DK7tH4XB6DJU53QvgE0wxBdBOEwuUU3kfo6CTI7tEgwALZcbJfiO_AD_mvxLxiogrIk9uJks5WUP3_-osL52U80wGMZH85tGgdvoh_GuRgmhk7BU30hCI1CD7euAQxlqWkJRiXyws3jptreTVJfPq7OxzQu6S8w03dLOgKIMx5t11MZKNyrnlklnVNDkVi",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 61.49923019999999,
                    "lng": 23.7756187
                },
                "viewport": {
                    "northeast": {
                        "lat": 61.50048722989271,
                        "lng": 23.77669507989272
                    },
                    "southwest": {
                        "lat": 61.49

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,30,gobabis,0,NaN,1558378567,24,-22.45,18.97,68.00,2.24,Sandune Game Lodge
1,166,brainerd,1,US,1558378527,40,46.36,-94.20,63.00,5.82,Grand View Lodge
2,168,straumen,0,NO,1558378786,52,63.87,11.30,68.00,9.17,"Radisson Blu Royal Garden Hotel, Trondheim"
3,192,port hedland,0,AU,1558378791,51,-20.31,118.58,62.60,6.93,Hedland Hotel
4,214,mount isa,0,AU,1558378794,56,-20.73,139.49,68.00,6.93,Isa Hotel
5,219,beisfjord,0,NO,1558378796,51,68.38,17.60,60.80,11.41,Hotell Fjället
6,230,moses lake,1,US,1558378804,22,47.13,-119.28,69.80,13.87,"Holiday Inn Express & Suites Moses Lake, an IH..."
7,259,hami,0,CN,1558378834,11,42.84,93.51,69.72,6.20,Hami Hotel
8,261,fort frances,1,CA,1558378834,38,48.61,-93.40,63.00,9.17,"Cantilever Hotel, Trademark Collection by Wyndham"
9,262,umtata,0,ZA,1558378835,51,-31.59,28.78,60.80,17.22,Ocean View Hotel


In [65]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]